**TXT to Json String**

In [43]:
import os
import json
import re

In [44]:
PATH_dir = '../data/ko_ner_data'
file_name_ko_train = 'train.txt'
file_name_ko_test = 'test.txt'
file_name_ko_dev = 'dev.txt'

In [45]:
file_name = file_name_ko_dev
PATH_data = os.path.join(PATH_dir, file_name)

In [46]:
pattern = re.compile('[IB]-[a-zA-Z]+')


In [47]:

with open(PATH_data, 'r') as f:
	
	fdata = f.read().split('\n\n')
	data = []

	for sentences in fdata[:-1]:

		sdata = []

		morph_sent = ""
		word_list = []
		ner_list = []

		sentence = sentences.split('\n')
		
		if sentence[-1] == "":
			sentence.pop()

		for line in sentence:
			components = line.split('\t')
			word = components[0].split("/")[0]

			if not (re.match(pattern, components[-1]) or components[-1] == 'O'):
				continue

			morph_sent += word + " "
			word_list.append(word)
			ner_list.append(components[-1])
		
		morph_sent = morph_sent[:-1]

		sdata.append(morph_sent)
		sdata.append(word_list)
		sdata.append(ner_list)
		
		data.append(sdata)

		
		
		

In [48]:

for sdata in data[:3]:
	print("morph_sent", sdata[0], "words", sdata[1], "ner", sdata[2], sep = '\n', end = '\n\n')

morph_sent
선방 한 정성룡 이 볼 을 걷 어 내 고 있 다 .
words
['선방', '한', '정성룡', '이', '볼', '을', '걷', '어', '내', '고', '있', '다', '.']
ner
['O', 'O', 'B-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

morph_sent
김학범 감독 이 장학영 에게 작전 지시 를 하 고 있 다 .
words
['김학범', '감독', '이', '장학영', '에게', '작전', '지시', '를', '하', '고', '있', '다', '.']
ner
['B-PS', 'O', 'O', 'B-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

morph_sent
또 박지성 은 지난 달 16 일 더비 카운티전 이후 4 경기 만 에 프리미어리그 경기 에 출장 , 지난 달 2 일 풀럼전 골 포함 시즌 두 번 째 공격 포인트 를 기록 했 다 .
words
['또', '박지성', '은', '지난', '달', '16', '일', '더비', '카운티전', '이후', '4', '경기', '만', '에', '프리미어리그', '경기', '에', '출장', ',', '지난', '달', '2', '일', '풀럼전', '골', '포함', '시즌', '두', '번', '째', '공격', '포인트', '를', '기록', '했', '다', '.']
ner
['O', 'B-PS', 'O', 'B-DT', 'I-DT', 'I-DT', 'I-DT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-OG', 'O', 'O', 'O', 'O', 'B-DT', 'I-DT', 'I-DT', 'I-DT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



**Tokenizing Json String**

In [49]:

from transformers import BertTokenizer

In [50]:
PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRETAINED_MODEL_NAME)

In [51]:
preprodata = []

for sdata in data:
	morph_to_tokens = tokenizer.tokenize(sdata[0])

	prepro_sdata = []
	tokenized_ner_list = []
	iterated_word = iter(sdata[1])
	iterated_ner = iter(sdata[2])

	word = ""
	entire_word = next(iterated_word)
	ner = next(iterated_ner)
	tokenized_ner_list.append(ner)

	for token in morph_to_tokens:

		if token[:2] == "##":
			word += token[2:]
		else:
			word += token
		
		if word == (entire_word or "[UNK]") and word != sdata[1][-1]:
			word = ""
			entire_word = next(iterated_word)
			ner = next(iterated_ner)
			tokenized_ner_list.append(ner)

		else:
			if ner == "O":
				tokenized_ner_list.append("O")
			else:
				tokenized_ner_list.append("I" + ner[1:])

	prepro_sdata.append(sdata[0])
	prepro_sdata.append(morph_to_tokens)
	prepro_sdata.append(tokenized_ner_list)

	
	preprodata.append(prepro_sdata)
	

In [52]:
print(preprodata[0][0])
print(preprodata[0][1])
print(preprodata[0][2])
print("\n")
print(preprodata[1][0])
print(preprodata[1][1])
print(preprodata[1][2])
print("\n")
print(preprodata[191][0])
print(preprodata[191][1])
print(preprodata[191][2])

선방 한 정성룡 이 볼 을 걷 어 내 고 있 다 .
['선', '##방', '한', '정', '##성', '##룡', '이', '볼', '을', '걷', '어', '내', '고', '있', '다', '.']
['O', 'O', 'O', 'B-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


김학범 감독 이 장학영 에게 작전 지시 를 하 고 있 다 .
['김', '##학', '##범', '감독', '이', '장', '##학', '##영', '에', '##게', '작', '##전', '지', '##시', '를', '하', '고', '있', '다', '.']
['B-PS', 'I-PS', 'I-PS', 'O', 'O', 'B-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


호지슨 감독 은 올해 초 설기현 과 말싸움 을 벌였 다 .
['호', '##지', '##슨', '감독', '은', '올', '##해', '초', '설', '##기', '##현', '과', '말', '##싸', '##움', '을', '벌', '##였', '다', '.']
['B-PS', 'I-PS', 'I-PS', 'O', 'O', 'B-DT', 'I-DT', 'I-DT', 'B-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [53]:
prepro_file_name = 'prepro_' + file_name.rsplit('.')[0]
PATH_preprodata = os.path.join(PATH_dir, prepro_file_name + '.json')

with open(PATH_preprodata, 'w') as f:
    json.dump(preprodata, f, ensure_ascii=False, indent='\t')

**Tag List**

In [54]:
prepro_tag_list = []
PATH_preprodata_tag_list = os.path.join(PATH_dir, prepro_file_name + '_tag_list.json')

with open(PATH_preprodata_tag_list, 'w') as f:

	for prepro_sdata in preprodata:
		prepro_tag_list.extend(prepro_sdata[2])
	
	prepro_tag_list = list(set(prepro_tag_list))

	json.dump(prepro_tag_list, f, ensure_ascii=False, indent='\t')


In [55]:

# prepro_tag_cnt_dict = {}

# for tag in prepro_tag_extend_list:
# 	try: prepro_tag_cnt_dict[tag] += 1
# 	except: prepro_tag_cnt_dict[tag] = 1